# 01. PyTorch Workflow Fundamentals

You: have some data + want it modelled.

Let's do that with PyTorch.

How about we start with a straight line?

And we see if we can build a model with PyTorch to predict that straight line.

## TK - What we're going to cover

1. Data
2. Load data
3. Build model
4. Fitting model to data (training)
5. Making predictions (inference)
6. Saving and loading model
7. Putting it all together

